In [9]:
%load_ext autoreload
%autoreload 2

In [1]:
from mingptf.model import GPT
import numpy as np
import tensorflow as tf
from mingptf.bpe import BPETokenizer
from transformers import GPT2Tokenizer, GPT2LMHeadModel

2023-01-09 20:36:48.895672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 20:36:48.895672: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/abhaykumar/opt/anaconda3/envs/mingpt/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/abhaykumar/opt/anaconda3/envs/mi

In [2]:
use_mingpt = True # use minGPT or huggingface/transformers model?
model_type = 'gpt2'

In [3]:
if use_mingpt:
    model = GPT.from_pretrained(model_type)
else:
    model = GPT2LMHeadModel.from_pretrained(model_type)
    model.config.pad_token_id = model.config.eos_token_id # suppress a warning


2023-01-09 20:36:56.848375: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-09 20:36:56.848375: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


number of parameters: 124.44M
number of parameters: 124.44M


All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


tfgpt2lm_head_model/transformer/wpe/embeddings:0 ---- gpt/wpe/embeddings:0
tfgpt2lm_head_model/transformer/wte/weight:0 ---- gpt/wte/embeddings:0
tfgpt2lm_head_model/transformer/wpe/embeddings:0 ---- gpt/wpe/embeddings:0
tfgpt2lm_head_model/transformer/wte/weight:0 ---- gpt/wte/embeddings:0
tfgpt2lm_head_model/transformer/h_._0/ln_1/gamma:0 ---- gpt/block/ln_1/gamma:0
tfgpt2lm_head_model/transformer/h_._0/ln_1/beta:0 ---- gpt/block/ln_1/beta:0
tfgpt2lm_head_model/transformer/h_._0/attn/c_attn/weight:0 ---- gpt/block/causal_self_attention/c_attn/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._0/attn/c_proj/weight:0 ---- gpt/block/causal_self_attention/c_proj/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._0/ln_2/gamma:0 ---- gpt/block/ln_2/gamma:0
tfgpt2lm_head_model/transformer/h_._0/ln_2/beta:0 ---- gpt/block/ln_2/beta:0
tfgpt2lm_head_model/transformer/h_._0/mlp/c_fc/weight:0 ---- gpt/block/mlp/c_fc/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._0/mlp/c_proj/weight:0 ---- gpt/bloc

0.0
tfgpt2lm_head_model/transformer/h_._5/attn/c_proj/weight:0 ---- gpt/block_5/causal_self_attention_5/c_proj/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._5/ln_2/gamma:0 ---- gpt/block_5/ln_2/gamma:0
tfgpt2lm_head_model/transformer/h_._5/ln_2/beta:0 ---- gpt/block_5/ln_2/beta:0
tfgpt2lm_head_model/transformer/h_._5/mlp/c_fc/weight:0 ---- gpt/block_5/mlp_5/c_fc/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._5/mlp/c_proj/weight:0 ---- gpt/block_5/mlp_5/c_proj/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._6/ln_1/gamma:0 ---- gpt/block_6/ln_1/gamma:0
tfgpt2lm_head_model/transformer/h_._6/ln_1/beta:0 ---- gpt/block_6/ln_1/beta:0
tfgpt2lm_head_model/transformer/h_._6/attn/c_attn/weight:0 ---- gpt/block_6/causal_self_attention_6/c_attn/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._6/attn/c_proj/weight:0 ---- gpt/block_6/causal_self_attention_6/c_proj/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._6/ln_2/gamma:0 ---- gpt/block_6/ln_2/gamma:0
tfgpt2lm_head_model/transformer/h_._6

tfgpt2lm_head_model/transformer/h_._10/mlp/c_fc/weight:0 ---- gpt/block_10/mlp_10/c_fc/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._10/mlp/c_proj/weight:0 ---- gpt/block_10/mlp_10/c_proj/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._11/ln_1/gamma:0 ---- gpt/block_11/ln_1/gamma:0
tfgpt2lm_head_model/transformer/h_._11/ln_1/beta:0 ---- gpt/block_11/ln_1/beta:0
tfgpt2lm_head_model/transformer/h_._11/attn/c_attn/weight:0 ---- gpt/block_11/causal_self_attention_11/c_attn/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._11/attn/c_proj/weight:0 ---- gpt/block_11/causal_self_attention_11/c_proj/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._11/ln_2/gamma:0 ---- gpt/block_11/ln_2/gamma:0
tfgpt2lm_head_model/transformer/h_._11/ln_2/beta:0 ---- gpt/block_11/ln_2/beta:0
tfgpt2lm_head_model/transformer/h_._11/mlp/c_fc/weight:0 ---- gpt/block_11/mlp_11/c_fc/kernel:0
0.0
tfgpt2lm_head_model/transformer/h_._11/mlp/c_proj/weight:0 ---- gpt/block_11/mlp_11/c_proj/kernel:0
0.0
tfgpt2lm_head_mode

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained(model_type)

In [5]:
tokenizer.tokenize("hi")

['hi']

['hi']

In [6]:
tokenizer = BPETokenizer()

In [7]:
tokenizer("hello")

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[31373]], dtype=int32)>

<tf.Tensor: shape=(1, 1), dtype=int32, numpy=array([[31373]], dtype=int32)>

In [8]:
model.generate(tokenizer, context="hei", max_new_tokens=10, temperature=1, top_k=8)

ValueError: Exception encountered when calling layer 'gpt' (type GPT).

too many values to unpack (expected 2)

Call arguments received by layer 'gpt' (type GPT):
  • x=tf.Tensor(shape=(1, 1, 1), dtype=int32)

ValueError: Exception encountered when calling layer 'gpt' (type GPT).

too many values to unpack (expected 2)

Call arguments received by layer 'gpt' (type GPT):
  • x=tf.Tensor(shape=(1, 1, 1), dtype=int32)

In [5]:
def generate(prompt='', num_samples=10, steps=20, do_sample=True):
        
    # tokenize the input prompt into integer input sequence
    if use_mingpt:
        tokenizer = BPETokenizer()
        if prompt == '':
            # to create unconditional samples...
            # manually create a tensor with only the special <|endoftext|> token
            # similar to what openai's code does here https://github.com/openai/gpt-2/blob/master/src/generate_unconditional_samples.py
            x = torch.tensor([[tokenizer.encoder.encoder['<|endoftext|>']]], dtype=torch.long)
        else:
            x = tokenizer(prompt).to(device)
    else:
        tokenizer = GPT2Tokenizer.from_pretrained(model_type)
        if prompt == '': 
            # to create unconditional samples...
            # huggingface/transformers tokenizer special cases these strings
            prompt = '<|endoftext|>'
        encoded_input = tokenizer(prompt, return_tensors='pt').to(device)
        x = encoded_input['input_ids']
    
    # we'll process all desired num_samples in a batch, so expand out the batch dim
    x = x.expand(num_samples, -1)

    # forward the model `steps` times to get samples, in a batch
    y = model.generate(x, max_new_tokens=steps, do_sample=do_sample, top_k=40)
    
    for i in range(num_samples):
        out = tokenizer.decode(y[i].cpu().squeeze())
        print('-'*80)
        print(out)
        

In [9]:
generate(prompt='Andrej Karpathy, the', num_samples=10, steps=20)

downloading https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json to /Users/abhaykumar/.cache/mingpt/encoder.json
downloading https://openaipublic.blob.core.windows.net/gpt-2/models/124M/encoder.json to /Users/abhaykumar/.cache/mingpt/encoder.json
downloading https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe to /Users/abhaykumar/.cache/mingpt/vocab.bpe
downloading https://openaipublic.blob.core.windows.net/gpt-2/models/124M/vocab.bpe to /Users/abhaykumar/.cache/mingpt/vocab.bpe


AttributeError: module 'tensorflow' has no attribute 'float'

AttributeError: module 'tensorflow' has no attribute 'float'